In [ ]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os


ModuleNotFoundError: No module named 'langchain_core'

In [ ]:
# Load
load_dotenv()

# Koneksi ke Qdrant
qdrant_client = QdrantClient(url="http://localhost:6333")

# Load embedding model yang sama
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load kembali vector store
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name="instru_collection",
    embeddings=embedding_model,
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
# Buat memory buffer
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Buat prompt template
template = """You are InstruRAG, an AI Agent specializing in Instrumentation and Control Engineering.
Use the provided context from the knowledge base to answer clearly and accurately.

Context:
{context}

Conversation History:
{chat_history}

Question:
{question}

Answer in a concise, technical, and easy-to-understand manner.
"""

rag_prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template=template
)

In [ ]:
# Buat conversational chain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(
        openai_api_key=os.getenv("GROQ_API_KEY"),
        openai_api_base="https://api.groq.com/openai/v1",
        model_name="llama-3.1-8b-instant",
        temperature=0.2,
    ),
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": rag_prompt}
)

In [ ]:
print("InstruRAG is ready!")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Goodbye!")
        break
    
    response = rag_chain.invoke({"question": user_input})
    print(f"InstruRAG: {response}\n")

In [ ]:
import uuid
from openai import OpenAI

# Client Groq LLM
client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

In [ ]:
def add_many_knowledge(texts: list):
    """Menambahkan banyak knowledge sekaligus ke Qdrant"""
    points = []
    for text in texts:
        embedding = embedding_model.embed_query(text)
        points.append({
            "id": str(uuid.uuid4()),
            "vector": embedding,
            "payload": {"page_content": str(text)}
        })
    
    qdrant_client.upsert(
        collection_name="instru_collection",
        points=points
    )
    return f"{len(texts)} knowledge entries added successfully."


In [ ]:
def retrieve_context(query: str) -> str:
    query_vector = embedding_model.embed_query(query)

    results = qdrant_client.query_points(
        collection_name="instru_collection",
        query=query_vector,
        limit=3
    )

    context = "\n\n".join([
        r.payload.get("page_content") or r.payload.get("text", "")
        for r in results.points
        if r.payload.get("page_content") or r.payload.get("text")
    ])
    return context

In [ ]:
def ask_instru_rag(query: str) -> str:
    context = retrieve_context(query)

    prompt = f"""
You are InstruRAG, an expert in Instrumentation and Control Engineering.
Use ONLY the context below to answer the question clearly and accurately.

Context:
{context}

Question:
{query}

Answer in a concise, technical, and easy-to-understand manner:
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    return response.choices[0].message.content

In [ ]:
automation_knowledge = [
    "A PLC (Programmable Logic Controller) is a rugged digital computer used for automation of electromechanical processes such as control of machinery on factory assembly lines.",
    "A DCS (Distributed Control System) is an automated control system that uses multiple controllers distributed throughout a plant to control complex processes with high reliability.",
    "SCADA (Supervisory Control and Data Acquisition) is a system of software and hardware used to monitor and control industrial processes remotely in real-time.",
    "An HMI (Human-Machine Interface) is a user interface or dashboard that connects operators to controllers like PLCs and DCS for process visualization and control.",
    "RTU (Remote Terminal Unit) is a microprocessor-controlled device that interfaces objects in the physical world to a SCADA or DCS system by transmitting telemetry data.",
    "A PLC executes logic in a cyclic manner known as scan cycle: reading inputs, executing logic, and updating outputs.",
    "DCS systems are typically used in continuous process industries such as oil refining, chemical plants, and power generation due to their scalability and fault tolerance.",
    "SCADA systems often integrate with PLCs and RTUs to provide centralized monitoring, alarms, and historical data logging.",
    "HMI systems display real-time data such as process values, alarms, and trends, allowing operators to monitor and interact with the system.",
    "A fieldbus is an industrial communication network that connects field devices such as sensors, actuators, and controllers in automation systems.",
    "Modbus is a widely used communication protocol in industrial automation for connecting PLCs, sensors, and control devices.",
    "Profibus and Profinet are industrial communication standards used to connect PLCs, distributed I/O, and field devices in automation.",
    "OPC (OLE for Process Control) is a standard interface for interoperability between different industrial hardware and software systems.",
    "Redundancy in DCS and SCADA ensures high availability by providing backup controllers, networks, and servers.",
    "Historian software in SCADA/DCS is used to store, compress, and retrieve process data for long-term analysis and reporting."
]

print(add_many_knowledge(automation_knowledge))


# Tanya jawab
queries = [
    "What is the difference between PLC and DCS?",
    "How does SCADA work in an industrial plant?",
    "What is the role of HMI in process automation?",
    "Why is redundancy important in DCS?",
    "What is Modbus and why is it used in automation?"
]

for q in queries:
    print("\nQuestion:", q)
    print("Answer:", ask_instru_rag(q))

In [ ]:
print(answer[:2000])